In [ ]:
!head train.csv

In [ ]:
import numpy as np

train = np.genfromtxt('train.csv', delimiter=',', names=True)
test  = np.genfromtxt('test.csv',  delimiter=',', names=True)
print(train.dtype.names)
print(test.dtype.names)

train = train.view(np.float).reshape(train.shape + (-1,))
test  = test.view(np.float).reshape(test.shape   + (-1,))
train = train.reshape(len(train), -1)
test  = test.reshape( len(test),  -1)

#First column is a dataset ID
#Last column is price
train_x = train[:, 1:-1]
train_y = train[:, -1]

#No test y!
test_x = test[:, 1:]

#Handle missing values
train_x = np.where(np.isnan(train_x), \
    np.ma.array(train_x, mask=np.isnan(train_x)).mean(axis = 0), train_x)
train_y = np.where(np.isnan(train_x).any(), \
    np.ma.array(train_y, mask=train_y).mean(axis = 0), train_y)   
test_x = np.where(np.isnan(test_x), \
    np.ma.array(test_x, mask=np.isnan(test_x)).mean(axis = 0), test_x) 



In [ ]:
#investigate


In [ ]:
#kfolds
KFOLDS    = 10
TR_SPLIT  = 0.9
SEED      = 0

from sklearn.ensemble import GradientBoostingRegressor as regressor
from sklearn.model_selection import KFold as kfold
from sklearn.linear_model import LinearRegression as linear_regression
from sklearn.metrics import mean_squared_error

parameters = {'n_estimators': 500,
              'max_depth': 3,
              'learning_rate': 0.01}


def experiment_kfold():
    kf = kfold(n_splits=KFOLDS, shuffle=True, random_state=SEED)

    var_selection = []
    for train_idx, test_idx in kf.split(train_x):
        #print(test_idx)
        train_x_kf = train_x[train_idx, :]
        train_y_kf = train_y[train_idx]
        test_x_kf  = train_x[test_idx, :]

        r = regressor(**parameters)
        #r = linear_regression()
        r.fit(train_x_kf, train_y_kf)
        r.predict(test_x_kf)
        s = r.score(train_x[test_idx, :], train_y[test_idx])
        print(s)

experiment_kfold()

def train():
    r = regressor(**parameters, random_state=SEED)
    r.fit(train_x, train_y)
    out = r.predict(test_x)
    return out   


In [ ]:
#Print to Kaggle format
test_prices = train()
print(len(test_prices))
assert(len(test_prices) == 1459)

with open("submission.csv", "w") as f:
    f.write("Id,SalePrice\n")
    for i, price in enumerate(test_prices):
        f.write(str(1461 + i) + "," + str(price))
        if i != len(test_prices) - 1:
            f.write("\n")

In [ ]:
!head sample_submission.csv 
!head submission.csv

!tail sample_submission.csv
!tail submission.csv